<a href="https://colab.research.google.com/github/JonMyles494/My-First-Repo/blob/main/Cipher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import string

### DO NOT MODIFY THIS FUNCTION ###
def load_words(file_name):
    '''
    file_name (string): the name of the file containing
    the list of words to load

    Returns: a list of valid words. Words are strings of lowercase letters.

    Depending on the size of the word list, this function may
    take a while to finish.
    '''
    print('Loading word list from file...')
    # inFile: file
    in_file = open(file_name, 'r')
    # line: string
    line = in_file.readline()
    # word_list: list of strings
    word_list = line.split()
    print('  ', len(word_list), 'words loaded.')
    in_file.close()
    return word_list

### DO NOT MODIFY THIS FUNCTION ###
def is_word(word_list, word):
    '''
    Determines if word is a valid word, ignoring
    capitalization and punctuation

    word_list (list): list of words in the dictionary.
    word (string): a possible word.

    Returns: True if word is in word_list, False otherwise

    Example:
    >>> is_word(word_list, 'bat') returns
    True
    >>> is_word(word_list, 'asdf') returns
    False
    '''
    word = word.lower()
    word = word.strip(" !@#$%^&*()-_+={}[]|\:;'<>?,./\"")
    return word in word_list

### DO NOT MODIFY THIS FUNCTION ###
def get_story_string():
    """
    Returns: a joke in encrypted text.
    """
    f = open("story.txt", "r")
    story = str(f.read())
    f.close()
    return story

WORDLIST_FILENAME = 'words.txt'

class Message(object):
    ### DO NOT MODIFY THIS METHOD ###
    def __init__(self, text):
        '''
        Initializes a Message object

        text (string): the message's text

        a Message object has two attributes:
            self.message_text (string, determined by input text)
            self.valid_words (list, determined using helper function load_words
        '''
        self.message_text = text
        self.valid_words = load_words(WORDLIST_FILENAME)

    ### DO NOT MODIFY THIS METHOD ###
    def get_message_text(self):
        '''
        Used to safely access self.message_text outside of the class

        Returns: self.message_text
        '''
        return self.message_text

    ### DO NOT MODIFY THIS METHOD ###
    def get_valid_words(self):
        '''
        Used to safely access a copy of self.valid_words outside of the class

        Returns: a COPY of self.valid_words
        '''
        return self.valid_words[:]

    def build_shift_dict(self, shift):
        '''
        Creates a dictionary that can be used to apply a cipher to a letter.
        The dictionary maps every uppercase and lowercase letter to a
        character shifted down the alphabet by the input shift. The dictionary
        should have 52 keys of all the uppercase letters and all the lowercase
        letters only.

        shift (integer): the amount by which to shift every letter of the
        alphabet. 0 <= shift < 26

        Returns: a dictionary mapping a letter (string) to
                 another letter (string).
        '''
        shift_dict = {}  # Initialize the dictionary to store the shifted strings
        lowercase = string.ascii_lowercase
        uppercase = string.ascii_uppercase

        # Shift the lowercase letters
        for i in range(len(lowercase)):
          shift_index = (i + shift) % 26  # Shift 1 and modulus 26 to keep inside domain of letters
          shift_dict[lowercase[i]] = lowercase[shift_index]

        # Shift the uppercase letters
        for i in range(len(uppercase)):
          shift_index = (i + shift) % 26  # Shift 1 and modulus 26 to keep inside domain of letters
          shift_dict[uppercase[i]] = uppercase[shift_index]

        # Return the dictionary
        return shift_dict

    def apply_shift(self, shift):
        '''
        Applies the Caesar Cipher to self.message_text with the input shift.
        Creates a new string that is self.message_text shifted down the
        alphabet by some number of characters determined by the input shift

        shift (integer): the shift with which to encrypt the message.
        0 <= shift < 26

        Returns: the message text (string) in which every character is shifted
             down the alphabet by the input shift
        '''
        shift_dict = self.build_shift_dict(shift)  # Build dictionary which has shifted letters
        encrypted_message = [
            shift_dict[char] if char in shift_dict else char  # Shift if char in dict
            for char in self.message_text
        ]

        return ''.join(encrypted_message)  # Join list into a final encrypted string

class PlaintextMessage(Message):
    def __init__(self, text, shift):
        '''
        Initializes a PlaintextMessage object

        text (string): the message's text
        shift (integer): the shift associated with this message

        A PlaintextMessage object inherits from Message and has five attributes:
            self.message_text (string, determined by input text)
            self.valid_words (list, determined using helper function load_words)
            self.shift (integer, determined by input shift)
            self.encrypting_dict (dictionary, built using shift)
            self.message_text_encrypted (string, created using shift)

        Hint: consider using the parent class constructor so less
        code is repeated
        '''
        # Use the parent class to initialize
        super().__init__(text)
        self.shift = shift
        self.encrypting_dict = self.build_shift_dict(shift)  # Dictionary of shifted letters
        self.message_text_encrypted = self.apply_shift(shift)  # Encrypted message


    def get_shift(self):
        '''
        Used to safely access self.shift outside of the class

        Returns: self.shift
        '''
        return self.shift

    def get_encrypting_dict(self):
        '''
        Used to safely access a copy self.encrypting_dict outside of the class

        Returns: a COPY of self.encrypting_dict
        '''
        return self.encrypting_dict.copy()

    def get_message_text_encrypted(self):
        '''
        Used to safely access self.message_text_encrypted outside of the class

        Returns: self.message_text_encrypted
        '''
        return self.message_text_encrypted

    def change_shift(self, shift):
        '''
        Changes self.shift of the PlaintextMessage and updates other
        attributes determined by shift (ie. self.encrypting_dict and
        message_text_encrypted).

        shift (integer): the new shift that should be associated with this message.
        0 <= shift < 26

        Returns: nothing
        '''
        self.shift = shift
        self.encrypting_dict = self.build_shift_dict(shift)  # Rebuild the shifted dictionary
        self.message_text_encrypted = self.apply_shift(shift)  # Update the encrypted message


class CiphertextMessage(Message):
    def __init__(self, text):
        '''
        Initializes a CiphertextMessage object

        text (string): the message's text

        a CiphertextMessage object has two attributes:
            self.message_text (string, determined by input text)
            self.valid_words (list, determined using helper function load_words)
        '''
        super().__init__(text)  # Initialize using the parent class

    def decrypt_message(self):
        '''
        Decrypt self.message_text by trying every possible shift value
        and find the "best" one. We will define "best" as the shift that
        creates the maximum number of real words when we use apply_shift(shift)
        on the message text. If s is the original shift value used to encrypt
        the message, then we would expect 26 - s to be the best shift value
        for decrypting it.

        Note: if multiple shifts are  equally good such that they all create
        the maximum number of you may choose any of those shifts (and their
        corresponding decrypted messages) to return

        Returns: a tuple of the best shift value used to decrypt the message
        and the decrypted message text using that shift value
        '''

        max_valid_words = 0
        best_shift = 0
        best_message = self.message_text

        # Convert valid_words list to a set for faster lookup
        valid_words_set = set(self.valid_words)

        # Test each shift from zero to 25
        for shift in range(26):
          decrypt_text = self.apply_shift(26 - shift)
          words = decrypt_text.split()
          valid_word_count = 0

          for word in words:
            if is_word(valid_words_set,word):
              valid_word_count += 1

          # Update if the most valid words a showing
          if valid_word_count > max_valid_words:
            max_valid_words = valid_word_count
            best_shift = 26 - shift
            best_message = decrypt_text

        # Return the tuple (best_shift,best_message)
        return (best_shift,best_message)

def decrypt_story():
  """
  Decrypts the story from the encrypted file.

  Returns:
  A tuple containing the best shift value and the decrypted story.

  """
  # Retreive the encrypted story
  encrypted_story = get_story_string()

  # Create a CiphertextMessage instance with the encrypted story
  ciphertext_message = CiphertextMessage(encrypted_story)

  # Decrypt the story by calling decrypt_message
  return ciphertext_message.decrypt_message()



#Example test case (PlaintextMessage)
plaintext = PlaintextMessage('hello', 2)
print('Expected Output: jgnnq')
print('Actual Output:', plaintext.get_message_text_encrypted())

#Example test case (CiphertextMessage)
ciphertext = CiphertextMessage('jgnnq')
print('Expected Output:', (24, 'hello'))
print('Actual Output:', ciphertext.decrypt_message())

# Decrypt story
print(decrypt_story())



Loading word list from file...
   55901 words loaded.
Expected Output: jgnnq
Actual Output: jgnnq
Loading word list from file...
   55901 words loaded.
Expected Output: (24, 'hello')
Actual Output: (24, 'hello')
Loading word list from file...
   55901 words loaded.
(16, 'Jack Florey is a mythical character created on the spur of a moment to help cover an insufficiently planned hack. He has been registered for classes at MIT twice before, but has reportedly never passed a class. It has been the tradition of the residents of East Campus to become Jack Florey for a few nights each year to educate incoming students in the ways, means, and ethics of hacking.')


In [2]:
%cd /content/drive/MyDrive

/content/drive/MyDrive
